In [ ]:
create_network_dict(1,4)

{'Layer1': {'node0': [], 'node1': [], 'node2': [], 'node3': []}}

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Hyperparameters
input_dim = 1
hidden_dim = 4
output_dim = 1
num_samples = 100
num_epochs = 100
learning_rate = 0.001

def create_network_dict(num_layers, num_nodes_per_layer):
    network_dict = {}
    for layer in range(1, num_layers + 1):
        layer_name = f"Layer{layer}"
        network_dict[layer_name] = {f"node{i}": [] for i in range(num_nodes_per_layer)}
    return network_dict

# Define the Model
class CustomModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.node_dict = create_network_dict(num_layers = 1, num_nodes_per_layer=hidden_dim)
        self.node_dict_testing = create_network_dict(num_layers = 1, num_nodes_per_layer=hidden_dim)
        self.layers = 1
        self.hidden_dim = hidden_dim
        self.testing = False


    def forward(self, x):
        x = torch.relu(self.fc1(x))
        if self.testing:
            for j in range(len(x[0])):
                self.node_dict_testing[f"Layer{self.layers}"][f"node{j}"].extend(x[:, j].detach().cpu().numpy())
        else:
            for j in range(len(x[0])):
                self.node_dict[f"Layer{self.layers}"][f"node{j}"].extend(x[:, j].detach().cpu().numpy())
        return  self.fc2(x)

# Generate Training Data
def generate_data(num_samples):
    X = np.random.rand(num_samples, 1) * 10
    y = 2 * X
    return X, y

# Training Pipeline
def train_model(model, X_train, y_train, num_epochs=100, learning_rate=0.01):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        inputs = torch.tensor(X_train).float()
        targets = torch.tensor(y_train).float()
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Extract Node Value Ranges
def extract_node_value_ranges(model):
    node_dict = model.node_dict
    interval_dict = create_network_dict(num_layers=model.layers, num_nodes_per_layer=model.hidden_dim)
    for layer, subdict in node_dict.items():
        for node, values in subdict.items():
            min_values = min(values)
            max_values = max(values)
            interval_dict[layer][node] = [min_values, max_values]
    return interval_dict

# Test on Unseen Data
def test_model(model, X_unseen, y_true):
    with torch.no_grad():
        model.eval()
        model.testing = True
        x_unseen_tensor = torch.tensor(X_unseen, dtype=torch.float32).clone().detach()
        y_pred = model(x_unseen_tensor)
        prediction_error = torch.norm(y_true - y_pred).item()
        print(f'True Output: {y_true}, Predicted Output: {y_pred.item()}, Prediction Error: {prediction_error}')
        print("\nNode values after training: \n")
        print(model.node_dict_testing)



         # Check if values are in the defined intervals
        for layer, subdict in model.node_dict_testing.items():
            for node, values in subdict.items():
                min_value, max_value = value_ranges[layer][node]
                are_values_in_interval = all(min_value <= value <= max_value for value in values)
                print(f'Layer {layer}, Node {node}: Values in Interval: {are_values_in_interval}')

#Main Execution


X_train, y_train = generate_data(num_samples)


model = CustomModel(input_dim, hidden_dim, output_dim)


train_model(model, X_train, y_train, num_epochs, learning_rate)


value_ranges = extract_node_value_ranges(model)


X_unseen = torch.tensor([[5.0]], dtype=torch.float32)  # Unseen input
y_true = torch.tensor([10.0], dtype=torch.float32)   # Ground truth for unseen input


test_model(model, X_unseen, y_true)




Epoch 10/100, Loss: 1.5403879880905151
Epoch 20/100, Loss: 0.15524280071258545
Epoch 30/100, Loss: 0.14911393821239471
Epoch 40/100, Loss: 0.14494958519935608
Epoch 50/100, Loss: 0.140901118516922
Epoch 60/100, Loss: 0.13696342706680298
Epoch 70/100, Loss: 0.13313353061676025
Epoch 80/100, Loss: 0.12940850853919983
Epoch 90/100, Loss: 0.1257859170436859
Epoch 100/100, Loss: 0.12226250767707825
True Output: tensor([10.]), Predicted Output: 10.148580551147461, Prediction Error: 0.14858055114746094

Node values after training: 

{'Layer1': {'node0': [5.469106], 'node1': [3.874779], 'node2': [0.0], 'node3': [6.1977983]}}
Layer Layer1, Node node0: Values in Interval: True
Layer Layer1, Node node1: Values in Interval: True
Layer Layer1, Node node2: Values in Interval: True
Layer Layer1, Node node3: Values in Interval: True


<ipython-input-70-77162e054849>:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_unseen_tensor = torch.tensor(X_unseen, dtype=torch.float32).clone().detach()


In [ ]:
value_ranges

{'Layer1': {'node0': [0.0, 10.986401],
  'node1': [0.22584103, 8.393031],
  'node2': [0.0, 0.0],
  'node3': [0.6380094, 11.656104]}}